In [1]:
cd yolov12

c:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12


In [2]:
!pip install -r requirements.txt
!pip install -e .

ERROR: flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl is not a supported wheel on this platform.


Obtaining file:///C:/Users/ShailAntani/Downloads/colab/Fire_And_Smoke_detection/yolov12
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for ultralytics (pyproject.toml): started
  Building editable for ultralytics (pyproject.toml): finished with status 'done'
  Created wheel for ultralytics: filename=ultralytics-8.3.63-0.editable-py3-none-any.whl size=20292 sha256=ca6106b09037bf93dcf6db3f8f36d47be02d259827444094aa2172af6baf0363
  Stored in directory: C:\Users\ShailAntani\AppData\Local\Temp\pip-ephem-wheel-cache-jcdgpd2n\wheels

In [5]:
from ultralytics import YOLO

model = YOLO(r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt")
results = model(r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial.jpg")
results[0].show()


image 1/1 C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial.jpg: 640x640 2 smokes, 1 fire, 106.2ms
Speed: 4.5ms preprocess, 106.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
!pip install python-telegram-bot opencv-python python-dotenv requests filelock

   ---------------------------------------- 0.0/652.1 kB ? eta -:--:--
   ---------------------------------------- 652.1/652.1 kB 6.2 MB/s eta 0:00:00


In [7]:
import cv2
import time
from ultralytics import YOLO
from config import Config
from notification_service import NotificationService

# 1. Initialize the Notification Service
# This will automatically load your settings from .env
service = NotificationService(Config)
print("✅ Notification Service Initialized")

# 2. Load your YOLO Model
# Replace 'yolov8n.pt' with the path to your trained model (e.g., 'best.pt')
model = YOLO('yolov8n.pt') 

# 3. Start Webcam (0 is usually the default webcam)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
else:
    print("📷 Webcam started. Press 'q' to exit.")

# Variables to control alert frequency (don't spam yourself!)
last_alert_time = 0
alert_cooldown = 15  # Seconds to wait before sending another alert

try:
    while True:
        success, frame = cap.read()
        if not success:
            break

        # Run YOLO detection
        results = model(frame, verbose=False)
        
        # Check if Fire or Smoke is detected
        # (Assuming your model has classes like 'fire' or 'smoke')
        detected_label = None
        
        for r in results:
            for box in r.boxes:
                # Get class name (e.g., 'fire', 'smoke', 'person')
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                confidence = float(box.conf[0])

                # CUSTOMIZE HERE: Which classes trigger the alert?
                # Example: Trigger if 'fire' is detected with > 50% confidence
                if class_name in ['fire', 'smoke', 'person'] and confidence > 0.5:
                    detected_label = class_name
                    
                    # Draw box on frame
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(frame, f"{class_name} {confidence:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # --- NOTIFICATION LOGIC ---
        if detected_label:
            current_time = time.time()
            # Only send alert if cooldown has passed
            if (current_time - last_alert_time) > alert_cooldown:
                print(f"🚨 sending alert for: {detected_label}")
                
                # Triggers the Telegram/WhatsApp notification
                service.send_alert(frame, detection=detected_label)
                
                last_alert_time = current_time
        # --------------------------

        # Show the video feed
        cv2.imshow("Fire Detection", frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Error: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()
    service.cleanup()
    print("🛑 System stopped.")

ModuleNotFoundError: No module named 'config'

In [3]:
%cd "C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection"

C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection


In [26]:
import cv2
import os
from ultralytics import YOLO
from config import Config
from notification_service import NotificationService

# 1. Initialize Notification Service
service = NotificationService(Config)
print("✅ Notification Service Initialized")

# 2. Define Paths (Using your paths)
MODEL_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt"
IMAGE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial.jpg"

# 3. Load Model & Image
try:
    print(f"🔄 Loading model from: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)
    
    print(f"🖼️ Reading image: {IMAGE_PATH}")
    # Read image using OpenCV so we can draw on it and send it
    frame = cv2.imread(IMAGE_PATH)
    
    if frame is None:
        raise FileNotFoundError(f"Could not read image at {IMAGE_PATH}")

    # 4. Run Detection
    results = model(frame)
    detected_label = None
    
    # Process results
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            confidence = float(box.conf[0])
            
            # CUSTOMIZE: Check for your specific class names
            # Adjust confidence threshold as needed (e.g., > 0.4)
            print(f"Found: {class_name} ({confidence:.2f})")
            
            if class_name in ['fire', 'smoke', 'Fire', 'Smoke']: 
                detected_label = class_name
                
                # logic to draw bounding box on 'frame' so the alert image shows the box
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 4)
                cv2.putText(frame, f"{class_name} {confidence:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 5. Send Notification if Detected
    if detected_label:
        print(f"🚨 ALERT Triggered: {detected_label} detected!")
        service.send_alert(frame, detection=detected_label)
        print("✅ Alert sent to Telegram.")
    else:
        print("✅ No fire or smoke detected in this image.")

    # Optional: Show result in notebook
    # import matplotlib.pyplot as plt
    # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # plt.show()

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    # Cleanup background threads
    service.cleanup()

WhatsApp alerts disabled: Missing credentials


✅ Notification Service Initialized
🔄 Loading model from: C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt
🖼️ Reading image: C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial.jpg

0: 640x640 2 smokes, 1 fire, 478.5ms
Speed: 20.0ms preprocess, 478.5ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)
Found: fire (0.80)
Found: smoke (0.65)
Found: smoke (0.36)
🚨 ALERT Triggered: smoke detected!
✅ Alert sent to Telegram.


In [27]:
import cv2
import os
from ultralytics import YOLO
# Ensure paths are correct as per previous steps
try: from config import Config
except: pass # fallback
from notification_service import NotificationService

# 1. Initialize
service = NotificationService(Config)

# 2. Paths
MODEL_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt"
IMAGE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial2.jpg"

try:
    model = YOLO(MODEL_PATH)
    frame = cv2.imread(IMAGE_PATH)
    
    if frame is None:
        raise FileNotFoundError(f"Image not found at {IMAGE_PATH}")

    # 4. Run Detection
    results = model(frame)
    
    # CHANGE: Use a Set to store unique detections (avoids duplicates like "Fire, Fire")
    detected_items = set() 
    
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            confidence = float(box.conf[0])
            
            print(f"Found: {class_name} ({confidence:.2f})")
            
            if class_name in ['fire', 'smoke', 'Fire', 'Smoke']: 
                # Add to our specific collection
                detected_items.add(class_name)
                
                # Visuals
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 4)
                cv2.putText(frame, f"{class_name} {confidence:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 5. Send Notification with Combined Label
    if detected_items:
        # Join them like "Fire and Smoke" or just "Fire"
        final_label = " and ".join(detected_items)
        
        print(f"🚨 ALERT Triggered: {final_label} Detected!")
        service.send_alert(frame, detection=final_label)
        print("✅ Alert sent to Telegram.")
    else:
        print("✅ No fire or smoke detected.")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    service.cleanup()

WhatsApp alerts disabled: Missing credentials



0: 384x640 1 smoke, 2 fires, 419.7ms
Speed: 11.1ms preprocess, 419.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Found: smoke (0.89)
Found: fire (0.71)
Found: fire (0.33)
🚨 ALERT Triggered: fire and smoke Detected!
✅ Alert sent to Telegram.


In [17]:
ls

 Volume in drive C is OS
 Volume Serial Number is 043C-64AB

 Directory of C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection

01/09/2026  01:29 PM    <DIR>          .
01/09/2026  12:59 PM    <DIR>          ..
01/09/2026  01:34 PM               294 .env
01/09/2026  01:29 PM    <DIR>          __pycache__
01/09/2026  01:10 PM               596 config.py
01/09/2026  01:45 PM    <DIR>          detected_events
01/09/2026  12:59 PM    <DIR>          models
01/09/2026  01:29 PM             8,413 notification_service.py
01/09/2026  12:59 PM           258,026 trial.jpg
01/09/2026  12:59 PM           739,498 trial2.jpg
01/09/2026  12:59 PM            28,744 trial3.avif
01/09/2026  12:59 PM            28,744 trial3.jpg
01/09/2026  12:59 PM           126,457 trial4.jpg
01/09/2026  12:59 PM            51,367 trial5(1).jpg
01/09/2026  12:59 PM            51,367 trial5.jpg
01/09/2026  01:45 PM            31,489 untitiled2.ipynb
01/09/2026  12:59 PM         2,343,056 Untitled0.ipynb
01/09/2

In [ ]:
#In this CPU takes 0.3 seconds per frame, ie 10s vid = 1.5 mins
import cv2
import os
import time
import numpy as np
from PIL import Image
from ultralytics import YOLO

# --- FIX START ---
# Correct way to handle fallback
try: 
    from config import Config
except ImportError: 
    class Config: 
        pass 
# --- FIX END ---

from notification_service import NotificationService

# ================= CONFIGURATION =================
# 1. Set your Source Path (Image OR Video)
# SOURCE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial.jpg" 
SOURCE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\wildfire.mp4"

# 2. Config
MODEL_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt"
ALERT_COOLDOWN = 15  # Seconds (Only applies to video)
CONFIDENCE_THRESHOLD = 0.4
# =================================================

# Initialize Service
service = NotificationService(Config)
last_alert_time = 0

def process_frame(frame, model, img_w, img_h):
    """Runs detection and draws smart visualization on a single frame"""
    results = model(frame, verbose=False)
    detected_items = set()
    
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            conf = float(box.conf[0])
            
            if class_name in ['fire', 'smoke', 'Fire', 'Smoke'] and conf > CONFIDENCE_THRESHOLD:
                detected_items.add(class_name)
                
                # --- Visualization (Smart Margins) ---
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # Clamp coordinates
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(img_w, x2), min(img_h, y2)
                
                # Dynamic Color: Fire=Red, Smoke=Gray/Orange
                color = (0, 0, 255) if 'fire' in class_name.lower() else (50, 50, 50)
                
                # Draw Box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                
                # Label positioning
                label = f"{class_name} {conf:.2f}"
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                
                # Smart Text: Move inside if at top edge
                if y1 - 25 < 0:
                    text_y = y1 + h + 10
                else:
                    text_y = y1 - 8
                
                text_x = min(x1, img_w - w - 2)
                
                # Text Background
                cv2.rectangle(frame, (text_x, text_y - h - 5), (text_x + w, text_y + 4), color, -1)
                cv2.putText(frame, label, (text_x, text_y), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                           
    return detected_items

try:
    print(f"🔄 Loading Model: {os.path.basename(MODEL_PATH)}")
    model = YOLO(MODEL_PATH)

    # --- Detect File Type ---
    # We check extension to decide mode
    ext = os.path.splitext(SOURCE_PATH)[1].lower()
    is_video = ext in ['.mp4', '.avi', '.mov', '.mkv', '.webm']
    
    if is_video:
        print(f"🎥 Video detected: {SOURCE_PATH}")
        cap = cv2.VideoCapture(SOURCE_PATH)
        if not cap.isOpened(): raise FileNotFoundError("Could not open video file.")
        
        while True:
            success, frame = cap.read()
            if not success: 
                print("✅ Video finished.")
                break
            
            h, w = frame.shape[:2]
            detections = process_frame(frame, model, w, h)
            
            if detections:
                curr_time = time.time()
                if (curr_time - last_alert_time) > ALERT_COOLDOWN:
                    final_lbl = " and ".join(detections)
                    print(f"🚨 ALERT: {final_lbl} at {curr_time:.0f}")
                    # Send async alert
                    service.send_alert(frame, detection=final_lbl)
                    last_alert_time = curr_time
            
            # Show preview (Optional for Notebooks)
            cv2.imshow("Detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): break
            
        cap.release()
        cv2.destroyAllWindows()

    else:
        # --- Image Mode ---
        print(f"🖼️ Image detected: {SOURCE_PATH}")
        frame = cv2.imread(SOURCE_PATH)
        
        # Fallback for AVIF/Others
        if frame is None:
            try:
                pil_img = Image.open(SOURCE_PATH)
                frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
            except: raise FileNotFoundError(f"Could not read {SOURCE_PATH}")
            
        h, w = frame.shape[:2]
        detections = process_frame(frame, model, w, h)
        
        if detections:
            final_lbl = " and ".join(detections)
            print(f"🚨 ALERT: {final_lbl} Detected!")
            service.send_alert(frame, detection=final_lbl)
            print("✅ Notification Sent.")
        else:
            print("✅ No Fire/Smoke detected.")

except Exception as e:
    print(f"❌ Error: {e}")
except KeyboardInterrupt:
    print("🛑 User interrupted.")
finally:
    try: service.cleanup()
    except: pass

WhatsApp alerts disabled: Missing credentials


🔄 Loading Model: best.pt
🎥 Video detected: C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\wildfire.mp4
🚨 ALERT: smoke at 1767950850
🚨 ALERT: smoke at 1767950866
🚨 ALERT: smoke at 1767950881
🛑 User interrupted.


In [ ]:
import cv2
import os
import time
import numpy as np
from PIL import Image
from ultralytics import YOLO

# Fallback for config
try: 
    from config import Config
except ImportError: 
    class Config: 
        pass 

from notification_service import NotificationService

# ================= CONFIGURATION =================
# SOURCE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial.jpg" 
SOURCE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\wildfire.mp4"

MODEL_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt"
ALERT_COOLDOWN = 15
CONFIDENCE_THRESHOLD = 0.4
FRAME_SKIP = 5  # Process only every 5th frame (Increase to speed up)
# =================================================

service = NotificationService(Config)
last_alert_time = 0

def process_frame(frame, model, img_w, img_h):
    """Runs detection and draws smart visualization on a single frame"""
    results = model(frame, verbose=False)
    detected_items = set()
    
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            conf = float(box.conf[0])
            
            if class_name in ['fire', 'smoke', 'Fire', 'Smoke'] and conf > CONFIDENCE_THRESHOLD:
                detected_items.add(class_name)
                
                # --- Visualization ---
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(img_w, x2), min(img_h, y2)
                
                color = (0, 0, 255) if 'fire' in class_name.lower() else (50, 50, 50)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                
                label = f"{class_name} {conf:.2f}"
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                
                if y1 - 25 < 0: text_y = y1 + h + 10
                else: text_y = y1 - 8
                
                text_x = min(x1, img_w - w - 2)
                cv2.rectangle(frame, (text_x, text_y - h - 5), (text_x + w, text_y + 4), color, -1)
                cv2.putText(frame, label, (text_x, text_y), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                           
    return detected_items

try:
    print(f"🔄 Loading Model: {os.path.basename(MODEL_PATH)}")
    model = YOLO(MODEL_PATH)
    
    ext = os.path.splitext(SOURCE_PATH)[1].lower()
    is_video = ext in ['.mp4', '.avi', '.mov', '.mkv', '.webm']
    
    if is_video:
        print(f"🎥 Video detected: {SOURCE_PATH}")
        cap = cv2.VideoCapture(SOURCE_PATH)
        
        # Get total frames to show progress
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"🎞️ Total Frames: {total_frames}. Processing every {FRAME_SKIP}th frame...")
        
        if not cap.isOpened(): raise FileNotFoundError("Could not open video file.")
        
        frame_count = 0
        
        while True:
            success, frame = cap.read()
            if not success: 
                print("\n✅ Video finished processing.")
                break
            
            # --- FRAME SKIPPING to speed up ---
            frame_count += 1
            if frame_count % FRAME_SKIP != 0:
                continue
            
            # Progress print every 20 processed frames
            if frame_count % (FRAME_SKIP * 10) == 0:
                print(f"Processing frame {frame_count}/{total_frames}...", end='\r')

            h, w = frame.shape[:2]
            detections = process_frame(frame, model, w, h)
            
            if detections:
                curr_time = time.time()
                if (curr_time - last_alert_time) > ALERT_COOLDOWN:
                    final_lbl = " and ".join(detections)
                    print(f"\n🚨 ALERT: {final_lbl} at frame {frame_count}")
                    service.send_alert(frame, detection=final_lbl)
                    last_alert_time = curr_time
            
            # Show preview
            cv2.imshow("Detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                print("\n🛑 User stopped video.")
                break
            
        cap.release()
        cv2.destroyAllWindows()

    else:
        # --- Image Mode ---
        print(f"🖼️ Image detected: {SOURCE_PATH}")
        frame = cv2.imread(SOURCE_PATH)
        if frame is None:
            pil_img = Image.open(SOURCE_PATH)
            frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
            
        h, w = frame.shape[:2]
        detections = process_frame(frame, model, w, h)
        
        if detections:
            final_lbl = " and ".join(detections)
            print(f"🚨 ALERT: {final_lbl} Detected!")
            service.send_alert(frame, detection=final_lbl)
        else:
            print("✅ No Fire/Smoke detected.")

except Exception as e:
    print(f"❌ Error: {e}")
except KeyboardInterrupt:
    # Graceful exit if you press 'Stop' in Notebook
    print("\n🛑 User interrupted kernel.")
finally:
    try: service.cleanup()
    except: pass

WhatsApp alerts disabled: Missing credentials


🔄 Loading Model: best.pt
🎥 Video detected: C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\test_video.mp4
🎞️ Total Frames: 0. Processing every 2th frame...
❌ Error: Could not open video file.


In [ ]:
import cv2
import os
from ultralytics import YOLO
# Ensure paths are correct as per previous steps
try: from config import Config
except: pass # fallback
from notification_service import NotificationService

# 1. Initialize
service = NotificationService(Config)

# 2. Paths
MODEL_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\yolov12\runs\detect\train13\weights\best.pt"
IMAGE_PATH = r"C:\Users\ShailAntani\Downloads\colab\Fire_And_Smoke_detection\trial4.jpg"

try:
    model = YOLO(MODEL_PATH)
    frame = cv2.imread(IMAGE_PATH)
    
    if frame is None:
        raise FileNotFoundError(f"Image not found at {IMAGE_PATH}")

    # 4. Run Detection
    results = model(frame)
    
    # CHANGE: Use a Set to store unique detections (avoids duplicates like "Fire, Fire")
    detected_items = set() 
    
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            confidence = float(box.conf[0])
            
            print(f"Found: {class_name} ({confidence:.2f})")
            
            if class_name in ['fire', 'smoke', 'Fire', 'Smoke']: 
                # Add to our specific collection
                detected_items.add(class_name)
                
                # Visuals
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 4)
                cv2.putText(frame, f"{class_name} {confidence:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 5. Send Notification with Combined Label
    if detected_items:
        # Join them like "Fire and Smoke" or just "Fire"
        final_label = " and ".join(detected_items)
        
        print(f"🚨 ALERT Triggered: {final_label} Detected!")
        service.send_alert(frame, detection=final_label)
        print("✅ Alert sent to Telegram.")
    else:
        print("✅ No fire or smoke detected.")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    service.cleanup()

WhatsApp alerts disabled: Missing credentials



0: 320x640 (no detections), 124.3ms
Speed: 5.4ms preprocess, 124.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
✅ No fire or smoke detected.
